<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [Assessment]:** RAG Evaluation</font>

<br>

Welcome to the last notebook of the course! In the previous notebook, you integrated a vector store solution into a RAG pipeline! In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

<br>

### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>

### **Questions To Think About:**

- As you go along, remember what our metrics actually represent. Should our pipeline pass these objectives? Is our judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for our use case?
- If we left the vectorstore-as-a-memory component in our chain, do you think it would still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance? 

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://www.nvidia.com/en-sg/training/instructor-led-workshops/building-rag-agents-with-llms/). If sharing this material, please give credit and link back to the original course.

<br>

### **Environment Setup:**

In [8]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/pairwise_string/). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [9]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 187 chunks

Sample Chunk:

Paper: AVS-Mamba: Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation

Summary: The essence of audio-visual segmentation (AVS) lies in locating and
delineating sound-emitting objects within a video stream. While
Transformer-based methods have shown promise, their handling of long-range
dependencies struggles due to quadratic computational costs, presenting a
bottleneck in complex scenarios. To overcome this limitation and facilitate
complex multi-modal comprehension with linear complexity, we introduce
AVS-Mamba, a selective state space model to address the AVS task. Our framework
incorporates two key components for video understanding and cross-modal
learning: Temporal Mamba Block for sequential video processing and
Vision-to-Audio Fusion Block for advanced audio-vision integration. Building on
this, we develop the Multi-scale Temporal Encoder, aimed at enhancing the
learning of visual features across scales, facilitating the perception of
intra- and inter-frame inf

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN

# Fill in the TODO to retrieve context from your docstore
context_getter = RunnableLambda(
    lambda d: docs2str(docstore.similarity_search(d["input"], k=3))
)

retrieval_chain = {"input": (lambda x: x)} | RunnableAssign({"context": context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller

# Fill in the TODO to generate the final answer from prompt + LLM
generator_chain = chat_prompt | llm

generator_chain = {"output": generator_chain} | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

Did you know that there's a concept called "Temporal Mamba" that's being used in deep learning to improve segmentation and classification tasks in various domains, including medical imaging?

In a paper titled "Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation" [1], the authors introduce a Temporal Mamba Block that enables fine-grained fusion across frames, leading to significant performance improvements. This is especially useful in tasks like audio-visual segmentation, where temporal dependencies play a crucial role.

But what's even more fascinating is that the Temporal Mamba concept has been explored in various other domains, such as medical imaging and remote sensing. For instance, in a paper titled "Vivim: a video vision mamba for medical video object segmentation" [2], the authors apply the Temporal Mamba idea to medical video object segmentation, achieving impressive results.

It's amazing to see how a concept like Temporal Mamba can have such a far-reachin

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [11]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How do the Mamba-based approaches in DM-Mamba and AVS-Mamba address the limitations of existing 
techniques in handling long-range dependencies, and what innovative components and architectures do they introduce 
to improve performance and efficiency in their respective tasks?

Answer: Both DM-Mamba and AVS-Mamba address the limitations of existing techniques in handling long-range 
dependencies by leveraging Mamba, a paradigm for long-range dependency modeling with linear complexity. DM-Mamba 
pioneers the use of Mamba in k-space learning, introducing a circular scanning strategy for spectrum unfolding and 
mitigating long-range forgetting. AVS-Mamba, on the other hand, incorporates a Temporal Mamba Block for sequential 
video processing and a Vision-to-Audio Fusion Block for cross-modal learning. Additionally, both approaches 
introduce innovative components such as the multi-scale Mamba architecture in DM-Mamba and the Modality Aggregation
Decoder and Contextual Integration Pyramid in AVS-Mamba, which facilitate the perception of intra- and inter-frame 
information and enable advanced audio-vision integration, respectively.

QA Pair 2

Question: What is the common thread between the Skip Mamba (Skimba) denoiser in monocular 3D semantic scene 
completion and the Temporal Mamba Block in audio-visual segmentation, and how is it utilized in both applications?

Answer: The Mamba framework, employed in both the Skip Mamba denoiser and the Temporal Mamba Block, offers a 
balanced solution by expanding the receptive field without incurring high memory costs, which is beneficial for 3D 
semantic scene completion and audio-visual segmentation. Specifically, it is used in the Skip Mamba denoising 
diffusion network for efficiently processing long-sequence data in 3D scene completion and in the Temporal Mamba 
Block for sequential video processing in audio-visual segmentation.

QA Pair 3

Question: What is the common thread between the Skip Mamba (Skimba) diffusion model presented in "Skip Mamba 
Diffusion for Monocular 3D Semantic Scene Completion" and the Temporal Mamba Block mentioned in "AVS-Mamba: 
Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation"?

Answer: Both models employ the concept of "Mamba" in their architecture, which suggests the use of a selective 
state space model to facilitate complex processing tasks. In the case of Skip Mamba, it is used for 3D semantic 
scene completion, while in AVS-Mamba, it is used for audio-visual segmentation. The term "Mamba" seems to be 
associated with advanced state space modeling techniques that enable efficient processing of sequential and 
multi-modal data.

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [12]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above

rag_answers = []
for i, q in enumerate(synth_questions):
    # 1) Parse out the question text from "Question: ..."
    question_text = q.replace("Question:", "").strip()

    # 2) Invoke your RAG chain with the parsed question
    rag_answer = rag_chain.invoke(question_text)

    # 3) Retrieve the final string answer from the chain's output
    #rag_answer = rag_answer_dict.get("output", "")

    # 4) Save the answer for later comparisons
    rag_answers.append(rag_answer)

    # 5) Print out results
    pprint2(f"QA Pair {i+1}", q, "", sep="\n") 
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How do the Mamba-based approaches in DM-Mamba and AVS-Mamba address the limitations of existing 
techniques in handling long-range dependencies, and what innovative components and architectures do they introduce 
to improve performance and efficiency in their respective tasks?

RAG Answer: Great question! So, you're wondering how Mamba-based approaches like DM-Mamba and AVS-Mamba tackle the 
challenges of existing techniques in handling long-range dependencies.

First, let's talk about what Mamba brings to the table. The Selective State Space Model, or Mamba for short, is an 
innovative framework that's particularly good at capturing long-range contexts with linear computational 
complexity. This is in contrast to traditional models that often struggle with long-range dependencies.

In AVS-Mamba, the researchers leverage Mamba's ability to process longer sequences in a single pass, which enables 
the exchange of multi-frame video features across various scales. This is a game-changer for audio-visual 
segmentation tasks, as it strengthens the connections between adjacent frames while minimizing interference from 
redundant information in temporally distant frames.

To address the limitations of existing techniques, DM-Mamba and AVS-Mamba introduce some innovative components and 
architectures. For instance, AVS-Mamba includes a Multi-scale Temporal Encoder that extracts features across scales
within and between frames. This allows the model to capture more nuanced and detailed representations of video 
features.

The Modality Aggregation Decoder is another key component of AVS-Mamba. This decoder features a Vision-to-Audio 
Fusion Block that seamlessly integrates visual data from diverse spatiotemporal contexts into the audio stream. 
This is a unique approach that enables the model to take advantage of the strengths of both visual and audio 
modalities.

Lastly, the Contextual Integration Pyramid is an effective component of AVS-Mamba that combines the visual and 
audio features at different scales to capture cross-scale temporal consistency. This innovative architecture allows
AVS-Mamba to achieve state-of-the-art performance in audio-visual segmentation tasks.

In summary, Mamba-based approaches like DM-Mamba and AVS-Mamba address the limitations of existing techniques by 
introducing innovative components and architectures that can handle long-range dependencies with greater efficiency
and effectiveness.

[1] Quote "[Quote from AVS-Mamba: Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation]"

QA Pair 2
Question: What is the common thread between the Skip Mamba (Skimba) denoiser in monocular 3D semantic scene 
completion and the Temporal Mamba Block in audio-visual segmentation, and how is it utilized in both applications?

RAG Answer: Based on the information I retrieved, it seems that both the Skimba denoiser in 3D semantic scene 
completion and the Temporal Mamba Block in audio-visual segmentation share a common architecture, which is 
essentially a modified version of the Mamba block.

As per the quote "The main difference between Skimba block and Mamba block (Li, Singh, and Grover 2024; Zhu et al. 
2024) is that Skimba is optimized for memory-efficient extraction of rich direct and indirect spatial features by 
three STM layers, whereas Mamba captures spatial information from various scan directions, requiring considerable 
memory."

The Temporal Mamba Block, however, doesn't seem to be directly mentioned in this quote. But, it seems that the 
Temporal Mamba Block (Ti-Mamba) is a variant of the Mamba Block, and it's designed for temporal information 
processing. 

In both applications, the modified Mamba blocks are used as a key component of the overall technique. They're 
utilized to capture spatial features or temporal information, and they're an essential part of the architecture. 
However, the specific implementation details might vary depending on the application and requirements.

Regarding the Semantic Block in Skimba, it's a separate type of block devised to handle spatial features before 
passing them to the Skimba Block. 

To directly answer your question, while the Skimba denoiser and Temporal Mamba Block have a common architectural 
thread through the Mamba Block, it seems they're optimized for different use cases, with Skimba focusing on 3D 
spatial features and the Temporal Mamba Block focused on temporal information.

QA Pair 3
Question: What is the common thread between the Skip Mamba (Skimba) diffusion model presented in "Skip Mamba 
Diffusion for Monocular 3D Semantic Scene Completion" and the Temporal Mamba Block mentioned in "AVS-Mamba: 
Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation"?

RAG Answer: Great question! After reviewing the documents, I found a common connection between the two. 

The key thread seems to be the "Mamba" concept itself. In the context of "Skip Mamba Diffusion for Monocular 3D 
Semantic Scene Completion", the Skip Mamba (Skimba) diffusion model leverages a Triple Mamba configuration, 
inspired by Yang et al. (2024). 

On the other hand, in "AVS-Mamba: Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation", the 
Temporal Mamba Block is a component of the AVS-Mamba model. 

While the specific applications and architectures differ, it appears that both documents utilize the Mamba concept,
with the Skip Mamba diffusion model in the first document and the Temporal Mamba Block in the second document. The 
Mamba concept might be a common thread across different research studies, but more investigation is needed to 
confirm this.

Do you need more clarification or would you like me to expand on this?

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [17]:
## TODO: Adapt this prompt for whichever LLM you're actually interested in using. 
## If it's llama, maybe system message would be good?
# Enhanced Evaluation Prompt with Comprehensive System Message and Multiple Examples
# Define the Evaluation Prompt with System Message and Clear Instructions
eval_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an expert evaluator tasked with comparing two answers to the same question. "
     "Your job is to assess which answer is better based on correctness, completeness, and consistency. "
     "Be strict in your evaluation, follow the instructions precisely, and ensure the output is concise and clear. "
     "Do not provide any commentary outside the requested output format."),
    
    ("user", 
     """INSTRUCTION:
You are given a Question-Answer pair. Your task is to compare the two provided answers:
- Answer 1 (Ground Truth): This answer is assumed to be correct and serves as the benchmark.
- Answer 2 (RAG Answer): This answer may or may not be true and should be evaluated against Answer 1.

### Evaluation Criteria:
1. Accuracy: Does Answer 2 provide correct information consistent with Answer 1?
2. Relevance: Does Answer 2 directly answer the question posed?
3. Consistency: Does Answer 2 avoid introducing contradictions or irrelevant information?
4. Quality: Does Answer 2 improve upon the phrasing, detail, or explanation provided in Answer 1?

### Scoring Rules:
- **[1]:** Answer 2 is incorrect, incomplete, irrelevant, or introduces inconsistencies.
- **[2]:** Answer 2 is correct, consistent, and improves upon Answer 1.

### Output Format:
Provide the evaluation strictly in the following format:

[Score] Justification

### Examples:

#### Example 1:
Question: What is the capital of Italy? Answer 1 (Ground Truth): Rome is the capital of Italy. Answer 2 (RAG Answer): Milan is the capital of Italy.

Evaluation: [1] The second answer is incorrect because the capital of Italy is Rome, not Milan.


#### Example 2:
Question: Explain the significance of photosynthesis in plants. Answer 1 (Ground Truth): Photosynthesis allows plants to convert sunlight into energy, producing glucose and oxygen as by-products. Answer 2 (RAG Answer): Photosynthesis is the process by which plants create energy using sunlight, carbon dioxide, and water, producing glucose and oxygen.

Evaluation: [2] The second answer is better as it provides a more complete explanation of photosynthesis while maintaining consistency with the ground truth.

#### Example 3:
Question: Who wrote the play 'Romeo and Juliet'? Answer 1 (Ground Truth): William Shakespeare wrote 'Romeo and Juliet.' Answer 2 (RAG Answer): The play 'Romeo and Juliet' was written by William Shakespeare.

Evaluation: [2] The second answer is better as it uses a clearer structure while maintaining the same factual correctness as the ground truth.


Now, evaluate the following Question-Answer pair:

{qa_trio}

EVALUATION:
""")
])

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How do the Mamba-based approaches in DM-Mamba and AVS-Mamba address the limitations of existing
techniques in handling long-range dependencies, and what innovative components and architectures do they introduce 
to improve performance and efficiency in their respective tasks?

Synth Answer: Answer: Both DM-Mamba and AVS-Mamba address the limitations of existing techniques in handling 
long-range dependencies by leveraging Mamba, a paradigm for long-range dependency modeling with linear complexity. 
DM-Mamba pioneers the use of Mamba in k-space learning, introducing a circular scanning strategy for spectrum 
unfolding and mitigating long-range forgetting. AVS-Mamba, on the other hand, incorporates a Temporal Mamba Block 
for sequential video processing and a Vision-to-Audio Fusion Block for cross-modal learning. Additionally, both 
approaches introduce innovative components such as the multi-scale Mamba architecture in DM-Mamba and the Modality 
Aggregation Decoder and Contextual Integration Pyramid in AVS-Mamba, which facilitate the perception of intra- and 
inter-frame information and enable advanced audio-vision integration, respectively.

RAG Answer: Great question! So, you're wondering how Mamba-based approaches like DM-Mamba and AVS-Mamba tackle the 
challenges of existing techniques in handling long-range dependencies.

First, let's talk about what Mamba brings to the table. The Selective State Space Model, or Mamba for short, is an 
innovative framework that's particularly good at capturing long-range contexts with linear computational 
complexity. This is in contrast to traditional models that often struggle with long-range dependencies.

In AVS-Mamba, the researchers leverage Mamba's ability to process longer sequences in a single pass, which enables 
the exchange of multi-frame video features across various scales. This is a game-changer for audio-visual 
segmentation tasks, as it strengthens the connections between adjacent frames while minimizing interference from 
redundant information in temporally distant frames.

To address the limitations of existing techniques, DM-Mamba and AVS-Mamba introduce some innovative components and 
architectures. For instance, AVS-Mamba includes a Multi-scale Temporal Encoder that extracts features across scales
within and between frames. This allows the model to capture more nuanced and detailed representations of video 
features.

The Modality Aggregation Decoder is another key component of AVS-Mamba. This decoder features a Vision-to-Audio 
Fusion Block that seamlessly integrates visual data from diverse spatiotemporal contexts into the audio stream. 
This is a unique approach that enables the model to take advantage of the strengths of both visual and audio 
modalities.

Lastly, the Contextual Integration Pyramid is an effective component of AVS-Mamba that combines the visual and 
audio features at different scales to capture cross-scale temporal consistency. This innovative architecture allows
AVS-Mamba to achieve state-of-the-art performance in audio-visual segmentation tasks.

In summary, Mamba-based approaches like DM-Mamba and AVS-Mamba address the limitations of existing techniques by 
introducing innovative components and architectures that can handle long-range dependencies with greater efficiency
and effectiveness.

[1] Quote "[Quote from AVS-Mamba: Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation]"

Synth Evaluation: [2] The second answer is better as it provides a comprehensive explanation of how Mamba-based 
approaches address limitations of existing techniques, introduces new components, and improves performance and 
efficiency in their respective tasks, consistently with the ground truth, and offers clarity and coherence in its 
narrative, although some minor required additional clarification on specific techniques of particular approaches.

Set 2

Question: Question: What is the common thread between the Skip Mamba (Skimba) denoiser in monocular 3D semantic 
scene completion and the Temporal Mamba Block in audio-visual segmentation, and how is it utilized in both 
applications?

...

Synth Evaluation: [1] The second answer is incorrect because it contains inaccuracies and inconsistencies. 
Specifically, it references a study by Zhu et al. 2024 that is not mentioned in the ground truth, and introduces a 
new component ("Semantic Block in Skimba") that is not mentioned in the answer 1. Additionally, it mentions that 
the Temporal Mamba Block is a variant of the Mamba Block, which contradicts the statement in the ground truth that 
both use the same framework but differ in their specific implementation. The second answer also discusses the Mamba
Block being used to capture spatial features in Skimba and temporal information in the Temporal Mamba Block, which 
is inconsistent with the ground truth's statement that both use a balanced solution to expand the receptive field 
without incurring high memory costs, suggesting that the answer is trying to correct the phrasing to make sense but
fails to do so consistently.

Set 3

Question: Question: What is the common thread between the Skip Mamba (Skimba) diffusion model presented in "Skip 
Mamba Diffusion for Monocular 3D Semantic Scene Completion" and the Temporal Mamba Block mentioned in "AVS-Mamba: 
Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation"?

Synth Answer: Answer: Both models employ the concept of "Mamba" in their architecture, which suggests the use of a 
selective state space model to facilitate complex processing tasks. In the case of Skip Mamba, it is used for 3D 
semantic scene completion, while in AVS-Mamba, it is used for audio-visual segmentation. The term "Mamba" seems to 
be associated with advanced state space modeling techniques that enable efficient processing of sequential and 
multi-modal data.

RAG Answer: Great question! After reviewing the documents, I found a common connection between the two. 

The key thread seems to be the "Mamba" concept itself. In the context of "Skip Mamba Diffusion for Monocular 3D 
Semantic Scene Completion", the Skip Mamba (Skimba) diffusion model leverages a Triple Mamba configuration, 
inspired by Yang et al. (2024). 

On the other hand, in "AVS-Mamba: Exploring Temporal and Multi-modal Mamba for Audio-Visual Segmentation", the 
Temporal Mamba Block is a component of the AVS-Mamba model. 

While the specific applications and architectures differ, it appears that both documents utilize the Mamba concept,
with the Skip Mamba diffusion model in the first document and the Temporal Mamba Block in the second document. The 
Mamba concept might be a common thread across different research studies, but more investigation is needed to 
confirm this.

Do you need more clarification or would you like me to expand on this?

Synth Evaluation: [2] The second answer is better as it directly addresses the question by identifying the common 
thread, which is the employment of the "Mamba" concept in both models, and provides more specific details regarding
the architectures and models, thus improving upon the phrasing and detail of the ground truth, while maintaining 
consistency with it.

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [18]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.6666666666666666


----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/stable/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/v0.1/docs/modules/agents/) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated recently.**
- **Make sure you don't have some old session of [`09_langserve.ipynb`](09_langserve.ipynb) already occupying the port. Your assessment requires you to implement the new `/retriever` and `/generator` endpoints!!**

**Objective:** On launch, [**`frontend/frontend_block.py`**](frontend/frontend_block.py) had several lines of code which trigger the course pass condition. Your objective is to invoke that series of commands by using your pipeline to pass the **Evaluation** check! Recall [`09_langserve.ipynb`](09_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NIM**](https://www.nvidia.com/en-us/ai/), which offers microservice spinup routines that can be deployed on local compute.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.
- [**LangGraph**](https://python.langchain.com/docs/langgraph/) is a framework for graph-based LLM orchestration, and is a natural next step forward for those interested in [multi-agent workflows](https://blog.langchain.dev/langgraph-multi-agent-workflows/).
- [**DSPy**](https://github.com/stanfordnlp/dspy), a flow engineering framework that allows you to optimize LLM orchestration pipelines based on empirical performance results.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>